In [8]:
import tensorflow as tf

import numpy as np
import os
import time

In [9]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [10]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [11]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [12]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [13]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [14]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [15]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [16]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [17]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [18]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [19]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [20]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [21]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [22]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [23]:
seq_length = 100

In [24]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [25]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [26]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [27]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [28]:
dataset = sequences.map(split_input_target)

In [29]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [30]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [31]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [32]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [33]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [35]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [37]:
sampled_indices

array([52,  4, 37, 47, 31, 44, 19, 11, 45, 40, 44, 40, 47, 39, 49, 20, 65,
        7, 29, 33, 56, 35, 56, 11,  8, 42, 32, 24, 36, 11, 46, 27, 28, 61,
       47, 62, 15, 33, 51, 33, 51, 60, 20, 34, 29, 53, 56, 51, 24, 43, 43,
        4, 35, 24, 55, 31, 28, 19,  6, 42, 17, 29, 29,  9, 52, 16, 24,  9,
       52, 27, 24, 14,  4,  6, 58,  4, 60, 53, 35, 17, 29, 35, 37, 43, 13,
       10, 28, 65,  9, 10, 27, 20, 62, 46, 12, 30, 34, 61, 16, 23])

In [38]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' true; for truth is truth\nTo the end of reckoning.\n\nDUKE VINCENTIO:\nAway with her! Poor soul,\nShe sp'

Next Char Predictions:
 b"m$XhReF:faeahZjGz,PTqVq:-cSKW:gNOvhwBTlTluGUPnqlKdd$VKpROF'cDPP.mCK.mNKA$'s$unVDPVXd?3Oz.3NGwg;QUvCJ"


In [39]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.188618, shape=(), dtype=float32)


In [41]:
tf.exp(example_batch_mean_loss).numpy()

65.931625

In [42]:
model.compile(optimizer='adam', loss=loss)

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [44]:
EPOCHS = 20

In [45]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 600s 3s/step - loss: 2.6770
Epoch 2/20
172/172 [==============================] - 589s 3s/step - loss: 1.9655
Epoch 3/20
172/172 [==============================] - 589s 3s/step - loss: 1.6905
Epoch 4/20
172/172 [==============================] - 589s 3s/step - loss: 1.5343
Epoch 5/20
172/172 [==============================] - 590s 3s/step - loss: 1.4384
Epoch 6/20
172/172 [==============================] - 590s 3s/step - loss: 1.3721
Epoch 7/20
172/172 [==============================] - 588s 3s/step - loss: 1.3210
Epoch 8/20
172/172 [==============================] - 594s 3s/step - loss: 1.2755
Epoch 9/20
172/172 [==============================] - 592s 3s/step - loss: 1.2346
Epoch 10/20
172/172 [==============================] - 593s 3s/step - loss: 1.1947
Epoch 11/20
172/172 [==============================] - 592s 3s/step - loss: 1.1530
Epoch 12/20
172/172 [==============================] - 592s 3s/step - loss: 1.1106
Epoch 13/20
1

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [48]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The worthy Mantua usurp'd,
Come hither to visit the fairest circums
of but best upon thy town, and other mints,
And raphed villain, redemption himself
To damnath my daughter my prophet wall:
Signior Hortensio, what's it is: and who
shall in nothing drubs and trust my happiness and patience.

QUEEN MARGARET:
Go, as it were, my lord.

PRINCE EDWARD:
And to beat back to us or true? is free of young
Prove a specious fiend? what's a man of all and heir?
Ol hollow Marcius.

MENENIUS:
You are abused!

SEBASTIAN:
By my tongue in your own well-ede-ting in hit
with trember fellow Isabel. I am sell-advised:
All truth, let me be put in beauty's valour,
In private is thy met as dance
there words: you know the care we must,
And turn in usural granted mining,
Which craves in a little be a practise.

HORTENSIO:
I say she'll say, he will repouse a lover may,
To civil what is the which you will wish them eyes,
Twriving it out of battle was wont to break,
Sit in his looking on a piece of birth,
Th

In [49]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe true knights, of such damned swords,\nTo with the last, what of that dead,\nTo win your friend their wills for thy effection\nWith all the Ledis o'er his deign. The king's noticies.\nMadam, I'l banish you at the thirm and, as thou rags' taxking\nThese a britten and command; and I think,\nTo do beto my dangerous sparcele.\n\nKING LEWIS XI:\nThen, good my friend, what strange man?\nWhy droys no joy? use and a subject see,\nBy this unmather, I will give them mo\ndefirely and statutes and wives Kate a drop;\nAppear in earnest hath no shalleign as every power,\nAnd bid my sweet oppose to question; but your glad and\nthat they are undo endury night.\n\nJOHN OF GAUNT:\nWhat, not a word? where ragest with a bird?\n\nLADY CAPULET:\nMy lord,\nAs foread men he knows Isabel's.\n\nANGELO:\nAnd so I had thought it hated and thank,\nTo strike at Pomfret bloodish father!\nLet him be gone, to't boltly; and did you life;\nThou wouldst he did withou? pray you, bid me leave\nyou 

In [50]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [51]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Nay, rather, well-see's bast end.

AUFIDIUS:
Why, what's this?

PAdE:
How long a throw of it.

CORI


In [52]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [53]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [54]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [55]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 635s 4s/step - loss: 2.7385


In [56]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1709
Epoch 1 Batch 50 Loss 2.0537
Epoch 1 Batch 100 Loss 1.9934
Epoch 1 Batch 150 Loss 1.8647

Epoch 1 Loss: 2.0094
Time taken for 1 epoch 628.60 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8483
Epoch 2 Batch 50 Loss 1.7714
Epoch 2 Batch 100 Loss 1.6666
Epoch 2 Batch 150 Loss 1.6097

Epoch 2 Loss: 1.7265
Time taken for 1 epoch 632.28 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6243
Epoch 3 Batch 50 Loss 1.6249
Epoch 3 Batch 100 Loss 1.5861
Epoch 3 Batch 150 Loss 1.5130

Epoch 3 Loss: 1.5629
Time taken for 1 epoch 631.34 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4873
Epoch 4 Batch 50 Loss 1.4776
Epoch 4 Batch 100 Loss 1.4628
Epoch 4 Batch 150 Loss 1.4308

Epoch 4 Loss: 1.4617
Time taken for 1 epoch 632.48 sec
_________________________________________________________________